In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

print(torch.__version__)

0.4.1


In [31]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        
        self.lstm1 = nn.LSTMCell(1, 5)
        self.lstm2 = nn.LSTMCell(5, 5)
        self.linear = nn.Linear(5, 1)
        
    def forward(self, inp, future = 0):
        outputs = []
        
        #print("input shape for forward function is : ", inp.size())
        #this is not a bad idea, batch size should be ALWAYS be inp.size(0)
        hidden_t = torch.zeros(inp.size(0) , 5 , dtype=torch.double) #97, 5
        c_t = torch.zeros(inp.size(0) , 5 , dtype=torch.double) #97, 5
        
        hidden_t2 = torch.zeros(inp.size(0) , 5 , dtype=torch.double) #same 
        c_t2 = torch.zeros(inp.size(0) , 5 , dtype=torch.double) #same
        
        for i, input_t in enumerate(inp.chunk(inp.size(1) , dim=1)): #chunck into 999 number of chunks
            #print("input chunck size: ", input_t.size()) #97 , 1
            
            
            #shape of input_t batch, input_size, which is 97, 1
            #shape of hiddent_t is 97, 5, shape of c_t is sam\
            hidden_t , c_t = self.lstm1(input_t , (hidden_t , c_t))
            hidden_t2 , c_t2 = self.lstm2(hidden_t , (hidden_t2 , c_t2))
            
            #print("shape of hidden_t2 right before linear layer is : " , hidden_t2.size()) #97, 5
            output = self.linear(hidden_t2) #batch size is 97, weirdo :/ 
            outputs += [output]
            
        for i in range(future):
            hidden_t, c_t = self.lstm1(output, (hidden_t, c_t))
            hidden_t2, c_t2 = self.lstm2(hidden_t, (hidden_t2, c_t2))
            
            output = self.linear(hidden_t2)
            outputs += [output]
            
        #todo 
        #print("Outputs right before stacking is : " , outputs)
        outputs = torch.stack(outputs, 1).squeeze(2)
        #print("Outputs right after stacking is : " , outputs)
        return outputs

In [32]:
if __name__ == "__main__":
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    data = torch.load('traindata.pt')
    
    #print(data)
    inp = torch.from_numpy(data[3: , :-1])
    print(inp.shape)
    
    target = torch.from_numpy(data[3: , 1:])
    print("target shape: ", target.shape)
    
    test_input = torch.from_numpy(data[:3 , :-1])
    test_target = torch.from_numpy(data[:3 , 1:])
    
    #shape should be 3, 999
    print("test shape", test_input.shape)
    seq = Sequence()
    
    seq.double()
    
    criterion = nn.MSELoss()
    opt = optim.LBFGS(seq.parameters(), lr= 0.8)
    
    for i in range(15):
        print("Step: ", i)
        def closure():
            opt.zero_grad()
            out = seq(inp)
            loss = criterion(out, target)
            loss.backward()
            return loss

        opt.step(closure)
        
        with torch.no_grad():
            future = 500
            pred = seq(test_input, future=future)
            
#             print("prediction is " , pred)
            loss = criterion(pred[ : , :-future] , test_target)
#             print("test_loss " , loss.item())
            y = pred.detach().numpy()
            
#             print("y is : " , y)
        
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(inp.size(1)), yi[:inp.size(1)], color, linewidth = 2.0)
            
            plt.plot(np.arange(inp.size(1), inp.size(1) + future), yi[inp.size(1):], color + ':', linewidth = 2.0)
        
#         print("y sample is " , y[0])
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        plt.savefig('predict%d.pdf'%i)
        plt.close()
        

torch.Size([97, 999])
target shape:  torch.Size([97, 999])
test shape torch.Size([3, 999])
Step:  0
prediction is  tensor([[-0.2645, -0.1778, -0.1169,  ...,  0.6230,  0.6230,  0.6230],
        [-0.4405, -0.4892, -0.4904,  ..., -0.8528, -0.8528, -0.8528],
        [-0.3905, -0.4240, -0.4497,  ...,  0.6230,  0.6230,  0.6230]],
       dtype=torch.float64)
test_loss  0.026160951524164203
y is :  [[-0.26449389 -0.17783193 -0.11689301 ...,  0.62301229  0.62301229
   0.62301229]
 [-0.44049948 -0.48918783 -0.49038154 ..., -0.85276244 -0.85276244
  -0.85276244]
 [-0.39052919 -0.42397948 -0.44965045 ...,  0.62301229  0.62301229
   0.62301229]]
y sample is  [-0.26449389 -0.17783193 -0.11689301 ...,  0.62301229  0.62301229
  0.62301229]
Step:  1
prediction is  tensor([[ 0.2535, -0.3492,  0.0858,  ..., -0.3679, -0.3679, -0.3679],
        [-0.4011, -0.6959, -0.2517,  ..., -0.3679, -0.3679, -0.3679],
        [-0.1904, -0.7148, -0.3069,  ..., -0.3679, -0.3679, -0.3679]],
       dtype=torch.float64)
tes